In [22]:
#r "nuget: Jinaga"
#r "nuget: Jinaga.Graphviz"

Installed Packages Jinaga, 0.11.2 Jinaga.Graphviz, 0.11.2

In [23]:
using Jinaga;
using Jinaga.Http;
using Jinaga.Graphviz;

In [24]:
class TokenAuthenticationProvider : IHttpAuthenticationProvider
{
    private string token;

    public TokenAuthenticationProvider(string token)
    {
        this.token = token;
    }

    public void SetRequestHeaders(System.Net.Http.Headers.HttpRequestHeaders headers)
    {
        headers.Add("Authorization", $"Bearer {token}");
    }

    public Task<bool> Reauthenticate()
    {
        return Task.FromResult(false);
    }
}

Create a file called `settings.json` and put the following settings into it:

```json
{
    "HttpEndpoint": "https://repdev.jinaga.com/....",
    "AuthorizationToken": "eyJh...."
}
```

In [25]:
#!value --from-file settings.json --name settings

In [26]:
#!share settings --from value

var parsedSettings = System.Text.Json.JsonSerializer.Deserialize<Dictionary<string, string>>(settings);

var jinagaClient = JinagaClient.Create(options =>
{
    options.HttpEndpoint = new Uri(parsedSettings["HttpEndpoint"]);
    options.HttpAuthenticationProvider = new TokenAuthenticationProvider(parsedSettings["AuthorizationToken"]);
});

In [27]:
[FactType("Blog.User.Name")]
public record UserName(User user, string value, UserName[] prior) { }

[FactType("Blog.Site")]
public record Site(User creator, DateTime createdAt) { }

[FactType("Blog.Site.Deleted")]
public record SiteDeleted(Site site, DateTime deletedAt) { }

[FactType("Blog.Site.Restored")]
public record SiteRestored(SiteDeleted deleted) { }

[FactType("Blog.Site.Name")]
public record SiteName(Site site, string value, SiteName[] prior) { }

[FactType("Blog.Site.Domain")]
public record SiteDomain(Site site, string value, SiteDomain[] prior) { }

[FactType("Blog.Post")]
public record Post(Site site, User author, DateTime createdAt) { }

[FactType("Blog.Post.Title")]
public record PostTitle(Post post, string value, PostTitle[] prior) { }

[FactType("Blog.Post.Deleted")]
public record PostDeleted(Post post, DateTime deletedAt) { }

[FactType("Blog.Post.Restored")]
public record PostRestored(PostDeleted deleted) { }

[FactType("Blog.Post.Publish")]
public record Publish(Post post, DateTime date) { }

In [28]:
Renderer.RenderTypes(typeof(Site), typeof(SiteName), typeof(SiteDomain), typeof(SiteDeleted), typeof(SiteRestored))

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 9.0.0 (20230911.1827)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- Blog.Site -->
 
 Blog.Site 
 
 Blog.Site 
 
<!-- Jinaga.User -->
 
 Jinaga.User 
 
 Jinaga.User 
 
<!-- Blog.Site->Jinaga.User -->
 
 Blog.Site->Jinaga.User 
 
 
 creator 
 
<!-- Blog.Site.Name -->
 
 Blog.Site.Name 
 
 Blog.Site.Name 
 
<!-- Blog.Site.Name->Blog.Site -->
 
 Blog.Site.Name->Blog.Site 
 
 
 site 
 
<!-- Blog.Site.Name->Blog.Site.Name -->
 
 Blog.Site.Name->Blog.Site.Name 
 
 
 prior 
 
<!-- Blog.Site.Domain -->
 
 Blog.Site.Domain 
 
 Blog.Site.Domain 
 
<!-- Blog.Site.Domain->Blog.Site -->
 
 Blog.Site.Domain->Blog.Site 
 
 
 site 
 
<!-- Blog.Site.Domain->Blog.Site.Domain -->
 
 Blog.Site.Domain->Blog.Site.Domain 
 
 
 prior 
 
<!-- Blog.Site.Deleted -->
 
 Blog.Site.Deleted 
 
 Blog.Site.Deleted 
 
<!-- Blog.Site.Deleted->Blog.Site -->
 
 Blog.Site.Deleted->Blog.Site 
 
 
 site 
 
<!-- Blog.Site.Restored -->
 
 Blog.Site.Restored 
 
 Blog.Site.Restored 
 
<!-- Blog.Site.Restored->Blog.Site.Deleted -->
 
 Blog.Site.Restored->Blog.Site.Deleted 
 
 
 deleted

In [29]:
var sitesForUser = Given<User>.Match((user, facts) =>
    from site in facts.OfType<Site>()
    where site.creator == user &&
        !facts.Any<SiteDeleted>(deleted => deleted.site == site &&
            !facts.Any<SiteRestored>(restored => restored.deleted == deleted))
    select new
    {
        id = jinagaClient.Hash(site),
        names =
            from name in facts.OfType<SiteName>()
            where name.site == site &&
                !facts.Any<SiteName>(next => next.prior.Contains(name))
            select name.value,
        domains =
            from domain in facts.OfType<SiteDomain>()
            where domain.site == site &&
                !facts.Any<SiteDomain>(next => next.prior.Contains(domain))
            select domain.value
    }
);

(var user, var profile) = await jinagaClient.Login();
var sites = await jinagaClient.Query(sitesForUser, user);

sites

index value 0 { id = zpHY9/lF8ieV3jFWH3YjchORWQ6V8Z7kCWoHdAapwiArLk2KhVpzerkpoSy0tx9fgWpND+psmmr6W8muDVlwpw==, names = System.Linq.Enumerable+<OfTypeIterator>d__66`1[System.String], domains = System.Linq.Enumerable+<OfTypeIterator>d__66`1[System.String] } id zpHY9/lF8ieV3jFWH3YjchORWQ6V8Z7kCWoHdAapwiArLk2KhVpzerkpoSy0tx9fgWpND+psmmr6W8muDVlwpw== names [ Edited in Polyglot on Android ] (values) [ Edited in Polyglot on Android ] domains [ qedcode.com ] (values) [ qedcode.com ] 1 { id = 5oMxkYWX18458RwWh613VCf7+L9GgecvPvKmQNNoxPJiqZO+8bremU2Ce4oYAd+CQy2ib8SuU5HwekI4ytl0hw==, names = System.Linq.Enumerable+<OfTypeIterator>d__66`1[System.String], domains = System.Linq.Enumerable+<OfTypeIterator>d__66`1[System.String] } id 5oMxkYWX18458RwWh613VCf7+L9GgecvPvKmQNNoxPJiqZO+8bremU2Ce4oYAd+CQy2ib8SuU5HwekI4ytl0hw== names [ Michael L Perry on iPhone ] (values) [ Michael L Perry on iPhone ] domains [ michaelperry.net ] (values) [ michaelperry.net ]

In [32]:
var siteFactsForUser = Given<User>.Match((user, facts) =>
    from site in facts.OfType<Site>()
    where site.creator == user &&
        !facts.Any<SiteDeleted>(deleted => deleted.site == site &&
            !facts.Any<SiteRestored>(restored => restored.deleted == deleted))
    select site
);

var siteNamesForUser = Given<User>.Match((user, facts) =>
    from site in facts.OfType<Site>()
    where site.creator == user &&
        !facts.Any<SiteDeleted>(deleted => deleted.site == site &&
            !facts.Any<SiteRestored>(restored => restored.deleted == deleted))
    from name in facts.OfType<SiteName>()
    where name.site == site &&
        !facts.Any<SiteName>(next => next.prior.Contains(name))
    select name
);

var siteDomainsForUser = Given<User>.Match((user, facts) =>
    from site in facts.OfType<Site>()
    where site.creator == user &&
        !facts.Any<SiteDeleted>(deleted => deleted.site == site &&
            !facts.Any<SiteRestored>(restored => restored.deleted == deleted))
    from domain in facts.OfType<SiteDomain>()
    where domain.site == site &&
        !facts.Any<SiteDomain>(next => next.prior.Contains(domain))
    select domain
);

var siteFacts = await jinagaClient.Query(siteFactsForUser, user);
var siteNames = await jinagaClient.Query(siteNamesForUser, user);
var siteDomains = await jinagaClient.Query(siteDomainsForUser, user);

jinagaClient.RenderFacts(siteFacts, siteNames, siteDomains)

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 9.0.0 (20230911.1827)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- AQ4Gbhfr+dVEb5/jfuRvTIDY+Po2i13e8paA+5GhBQMiIj8fo1Y767RTZtuoxrURnb+W8VX8QtgBARPAOpZ59g== -->
 
 AQ4Gbhfr+dVEb5/jfuRvTIDY+Po2i13e8paA+5GhBQMiIj8fo1Y767RTZtuoxrURnb+W8VX8QtgBARPAOpZ59g== 
 
 Jinaga.User 
 
 publicKey 
 
 -----BEGIN PUBLIC KE... 
 
<!-- zpHY9/lF8ieV3jFWH3YjchORWQ6V8Z7kCWoHdAapwiArLk2KhVpzerkpoSy0tx9fgWpND+psmmr6W8muDVlwpw== -->
 
 zpHY9/lF8ieV3jFWH3YjchORWQ6V8Z7kCWoHdAapwiArLk2KhVpzerkpoSy0tx9fgWpND+psmmr6W8muDVlwpw== 
 
 Blog.Site 
 
 createdAt 
 
 2024-02-19T03:51:21.... 
 
 
<!-- zpHY9/lF8ieV3jFWH3YjchORWQ6V8Z7kCWoHdAapwiArLk2KhVpzerkpoSy0tx9fgWpND+psmmr6W8muDVlwpw==->AQ4Gbhfr+dVEb5/jfuRvTIDY+Po2i13e8paA+5GhBQMiIj8fo1Y767RTZtuoxrURnb+W8VX8QtgBARPAOpZ59g== -->
 
 zpHY9/lF8ieV3jFWH3YjchORWQ6V8Z7kCWoHdAapwiArLk2KhVpzerkpoSy0tx9fgWpND+psmmr6W8muDVlwpw==->AQ4Gbhfr+dVEb5/jfuRvTIDY+Po2i13e8paA+5GhBQMiIj8fo1Y767RTZtuoxrURnb+W8VX8QtgBARPAOpZ59g== 
 
 
 creator 
 
<!-- 5oMxkYWX18458RwWh613VCf7+L9GgecvPvKmQNNoxPJiqZO+8bremU2Ce4oYAd+CQy2ib8SuU5HwekI4ytl0hw== -->
 
 5oMxkYWX18458RwWh613VCf7+L9GgecvPvKmQNNoxPJiqZO+8bremU2Ce4oYAd+CQy2ib8SuU5HwekI4ytl0hw== 
 
 Blog.Site 
 
 createdAt 
 
 2024-02-19T03:51:58.... 
 
 
<!-- 5oMxkYWX18458RwWh613VCf7+L9GgecvPvKmQNNoxPJiqZO+8bremU2Ce4oYAd+CQy2ib8SuU5HwekI4ytl0hw==->AQ4Gbhfr+dVEb5/jfuRvTIDY+Po2i13e8paA+5GhBQMiIj8fo1Y767RTZtuoxrURnb+W8VX8QtgBARPAOpZ59g== -->
 
 5oMxkYWX18458RwWh613VCf7+L9GgecvPvKmQNNoxPJiqZO+8bremU2Ce4oYAd+CQy2ib8SuU5HwekI4ytl0hw==->AQ4Gbhfr+dVEb5/jfuRvTIDY+Po2i13e8paA+5GhBQMiIj8fo1Y767RTZtuoxrURnb+W8VX8QtgBARPAOpZ59g== 
 
 
 creator 
 
<!-- +gflKycBPHvdr0I6DE6Dff//s7ZVzseGu6i0aGJThuKIK0bTd+o79GUN2kc8fABEkfrTbFghTYGnTRxrZ+MRdg== -->
 
 +gflKycBPHvdr0I6DE6Dff//s7ZVzseGu6i0aGJThuKIK0bTd+o79GUN2kc8fABEkfrTbFghTYGnTRxrZ+MRdg== 
 
 Blog.Site.Name 
 
 value 
 
 Q.E.D. Code 
 
<!-- +gflKycBPHvdr0I6DE6Dff//s7ZVzseGu6i0aGJThuKIK0bTd+o79GUN2kc8fABEkfrTbFghTYGnTRxrZ+MRdg==->zpHY9/lF8ieV3jFWH3YjchORWQ6V8Z7kCWoHdAapwiArLk2KhVpzerkpoSy0tx9fgWpND+psmmr6W8muDVlwpw== -->
 
 +gflKycBPHvdr0I6DE6Dff//s7ZVzseGu6i0aGJThuKIK0bTd+o79GUN2kc8fABEkfrTbFghTYGnTRxrZ+MRdg==->zpHY9/lF8ieV3jFWH3YjchORWQ6V8Z7kCWoHdAapwiArLk2KhVpzerkpoSy0tx9fgWpND+psmmr6W8muDVlwpw== 
 
 
 site 
 
<!-- 0uhFkZ1/SvyY8CkXrcE0EHTREl/Zvucq4r1ULURO+h/RcsWvQJZTPP/M/MFqKPjEf6pwcODDkP/W8M5OlHu9DA== -->
 
 0uhFkZ1/SvyY8CkXrcE0EHTREl/Zvucq4r1ULURO+h/RcsWvQJZTPP/M/MFqKPjEf6pwcODDkP/W8M5OlHu9DA== 
 
 Blog.Site.Name 
 
 value 
 
 Mike’s Cool Blog 
 
<!-- 0uhFkZ1/SvyY8CkXrcE0EHTREl/Zvucq4r1ULURO+h/RcsWvQJZTPP/M/MFqKPjEf6pwcODDkP/W8M5OlHu9DA==->zpHY9/lF8ieV3jFWH3YjchORWQ6V8Z7kCWoHdAapwiArLk2KhVpzerkpoSy0tx9fgWpND+psmmr6W8muDVlwpw== -->
 
 0uhFkZ1/SvyY8CkXrcE0EHTREl/Zvucq4r1ULURO+h/RcsWvQJZTPP/M/MFqKPjEf6pwcODDkP/W8M5OlHu9DA==->zpHY9/lF8ieV3jFWH3YjchORWQ6V8Z7kCWoHdAapwiArLk2KhVpzerkpoSy0tx9fgWpND+psmmr6W8muDVlwpw== 
 
 
 site 
 
<!-- 0uhFkZ1/SvyY8CkXrcE0EHTREl/Zvucq4r1ULURO+h/RcsWvQJZTPP/M/MFqKPjEf6pwcODDkP/W8M5OlHu9DA==->+gflKycBPHvdr0I6DE6Dff//s7ZVzseGu6i0aGJThuKIK0bTd+o79GUN2kc8fABEkfrTbFghTYGnTRxrZ+MRdg== -->
 
 0uhFkZ1/SvyY8CkXrcE0EHTREl/Zvucq4r1ULURO+h/RcsWvQJZTPP/M/MFqKPjEf6pwcODDkP/W8M5OlHu9DA==->+gflKycBPHvdr0I6DE6Dff//s7ZVzseGu6i0aGJThuKIK0bTd+o79GUN2kc8fABEkfrTbFghTYGnTRxrZ+MRdg== 
 
 
 prior 
 
<!-- T/Ht8Ta3BD7ECOSG06QyNxTYNQy+7CWCNDKN2UpMBBwYUAul4hoHMuVTT8mwTwQ+EmcgeaLAqKQmVQ5vUiQ5yg== -->
 
 T/Ht8Ta3BD7ECOSG06QyNxTYNQy+7CWCNDKN2UpMBBwYUAul4hoHMuVTT8mwTwQ+EmcgeaLAqKQmVQ5vUiQ5yg== 
 
 Blog.Site.Name 
 
 value 
 
 Q. E. D. Code 
 
<!-- T/Ht8Ta3BD7ECOSG06QyNxTYNQy+7CWCNDKN2UpMBBwYUAul4hoHMuVTT8mwTwQ+EmcgeaLAqKQmVQ5vUiQ5yg==->zpHY9/lF8ieV3jFWH3YjchORWQ6V8Z7kCWoHdAapwiArLk2KhVpzerkpoSy0tx9fgWpND+psmmr6W8muDVlwpw== -->
 
 T/Ht8Ta3BD7ECOSG06QyNxTYNQy+7CWCNDKN2UpMBBwYUAul4hoHMuVTT8m